In [1]:
import os

import keras.metrics
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, UpSampling2D, Concatenate, Input
from tensorflow.keras.models import Model
import pandas as pd
from sklearn.model_selection import train_test_split
import utilities
from utilities import DataGenerator

In [2]:
os.chdir('data/Healthcare AI Datasets/Brain_MRI')
brain_df = pd.read_csv('data_mask.csv')

In [3]:
brain_df_mask = brain_df[brain_df['mask'] == 1]
brain_df_mask.shape

X_train, X_val = train_test_split(brain_df_mask, test_size=0.15)
X_test, X_val = train_test_split(X_val, test_size=0.5)

train_ids = list(X_train.image_path)
train_mask = list(X_train.mask_path)

val_ids = list(X_val.image_path)
val_mask= list(X_val.mask_path)

training_generator = DataGenerator(train_ids,train_mask)
validation_generator = DataGenerator(val_ids,val_mask)

In [4]:
def batchnorm_relu(inputs):
    x = BatchNormalization()(inputs)
    x = Activation("relu")(x)

    return x

def residual_block(inputs, num_filters, strides=1):
    # Convolutional Layers
    x = batchnorm_relu(inputs)
    x = Conv2D(num_filters, 3, padding="same", strides=strides)(x)
    x = batchnorm_relu(x)
    x = Conv2D(num_filters, 3, padding="same", strides=1)(x)

    # Shortcut Connection (Identity Mapping)
    s = Conv2D(num_filters, 1, padding="same", strides=strides)(inputs)

    x = x + s
    return x


def decoder_block(inputs, skip_features, num_filters):
    x = UpSampling2D((2, 2))(inputs)
    x = Concatenate()([x, skip_features])
    x = residual_block(x, num_filters, strides=1)
    return x


def build_resunet(input_shape):
    inputs = Input(input_shape)

    # Endoder 1
    x = Conv2D(64, 3, padding="same", strides=1)(inputs)
    x = batchnorm_relu(x)
    x = Conv2D(64, 3, padding="same", strides=1)(x)
    s = Conv2D(64, 1, padding="same")(inputs) # Skip connection
    s1 = x + s

    # Encoder 1 & 2
    s2 = residual_block(s1, 128, strides=2)
    s3 = residual_block(s2, 256, strides=2)

    # Bridge
    b = residual_block(s3, 512, strides=2)

    # Decoders
    x = decoder_block(b, s3, 256)
    x = decoder_block(x, s2, 128)
    x = decoder_block(x, s1, 64)

    # Classifier
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(x)

    #Model
    model = Model(inputs, outputs, name="RESUNET")

    return model

shape = (256, 256, 3)
model = build_resunet(shape)
model.summary()

Model: "RESUNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                           

In [5]:
model.compile(optimizer='adam',
            loss=utilities.tversky_loss,
            metrics=['precision'])

In [6]:
history = model.fit(training_generator,
                    epochs = 10,
                    validation_data=validation_generator)

Epoch 1/10


TypeError: in user code:

    File "C:\Users\dapal\PycharmProjects\tumor_detector\venv\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\dapal\PycharmProjects\tumor_detector\utilities.py", line 215, in tversky_loss  *
        return 1 - tversky(y_true,y_pred)
    File "C:\Users\dapal\PycharmProjects\tumor_detector\utilities.py", line 208, in tversky  *
        true_pos = K.sum(y_true_pos * y_pred_pos)

    TypeError: Input 'y' of 'Mul' Op has type float32 that does not match type int32 of argument 'x'.


In [9]:
training_generator.__getitem__(1)

(array([[[[-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942],
          ...,
          [-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942]],
 
         [[-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942],
          ...,
          [-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942]],
 
         [[-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942],
          ...,
          [-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942],
          [-0.73115752, -0.73115752, -0.70381942]],
 
         ...,
 
         [[-